First import relevant libraries. For wavelet transform we have used [PyWavelets package](https://pywavelets.readthedocs.io/en/latest/). Image resizing can be done in many ways. We have used Tensorflow for the same.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pywt
import glob

In [2]:
files = glob.glob('D:/matlab/case/process/48k/load_1/*.csv')
files

['D:/matlab/case/process/48k/load_1\\DE_48k_B007_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_B014_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_B021_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_IR007_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_IR014_1_X217_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_IR021_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_OR007_6_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_OR014_6_1.csv',
 'D:/matlab/case/process/48k/load_1\\DE_48k_OR021_6_1.csv',
 'D:/matlab/case/process/48k/load_1\\Normal_1.csv']

These files are there in the folder. We read each file sequentially, segmet it into lengths of 1024 and apply wavelet transform to it. Then we resize the wavelet ouput to size $(32 \times 32)$. Following code can be used to genrate data.

In [ ]:
files = glob.glob('D:/matlab/case/process/48k/load_1/*.csv')
data = np.empty((460*10,32,32))
data[:] = np.nan

count = 0
for file in files:
    temp = pd.read_csv(file, header = None)
    for j in range(460):
        segment = temp[0][(j*1024):((j+1)*1024)].values
        coefs,_ = pywt.cwt(segment,np.arange(1,2049,32), 'morl')
        data[count,:,:] = tf.reshape(tf.image.resize(coefs.reshape((64,1024,1)),(32,32)),(32,32))
        if (count % 100) == 0:
            print(f"{count} data points extracted")
        count = count + 1
fault_types = ['Ball_007', 'Ball_014', 'Ball_021', 'IR_007', 'IR_014', 'IR_021', 'OR_007','OR_014', 'OR_021', 'Normal']
labels = np.repeat(fault_types, 460)
# Save wavelet data
np.savez('CWRU_48k_load_1_CNN_wavelet_data', data = data, labels = labels)